# GAN Testing

* [YouTube | Generative Adversial Network (GANs) Full Coding Example Tutorial in Tensorflow 2.0!](https://www.youtube.com/watch?v=tX-6CMNnT64)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0, '../')
import data as d

## Settings

In [2]:
BATCHSIZE = 64
SEQLEN = 100
BUFFER = 10000
EMBED_DIM = 50
EPOCHS = 2
LATENT_UNITS = 100

## Data Preprocessing

In [3]:
# import The Meditations by Marcus Aurelius
txt = d.meditations()
print(txt[:200])

We have 507 stoic lessons from Marcus Aurelius
From my grandfather Verus I learned good morals and the government of my temper.
From the reputation and remembrance of my father, modesty and a manly character.
From my mother, piety and beneficence,


In [4]:
def format_data(txt):
    vocab = sorted(set(txt))  # create vocab from text string (txt)
    char2idx = {c: i for i, c in enumerate(vocab)}

    data_idx = np.array([char2idx[c] for c in txt])
    
    # one hot encode
    hot1 = np.zeros((data_idx.size, data_idx.max()+1))
    hot1[np.arange(data_idx.size), data_idx] = 1
    print(f"hot1 = {hot1.shape}")
    # and flatten
    hot1 = hot1.flatten()
    print(f"hot1 flatten = {hot1.shape}")
    
    # max length sequence we can have
    max_seqlen = (len(txt) * (data_idx.max()+1)) // (SEQLEN)
    
    dataset = tf.data.Dataset.from_tensor_slices(hot1)
    
    sequences = dataset.batch(SEQLEN*(data_idx.max()+1), drop_remainder=True)
    
    dataset = sequences.shuffle(BUFFER).batch(BATCHSIZE, drop_remainder=True)
    
    return dataset, char2idx

In [5]:
data, char2idx = format_data(txt)

hot1 = (241387, 67)
hot1 flatten = (16172929,)


## Models
Useful Links:
* [Medium | Music generation with Neural Networks](https://medium.com/cindicator/music-generation-with-neural-networks-gan-of-the-week-b66d01e28200)
* [arXiv | C-RNN-GAN: Continuous recurrent neural networks with adversial training](https://arxiv.org/pdf/1611.09904.pdf)
* [Medium | Generating Pokemon-Inspired Musiic from Neural Networks](https://towardsdatascience.com/generating-pokemon-inspired-music-from-neural-networks-bc240014132)

For the loss functions, we will use categorical cross entropy for both as both LSTM and Generative models can use it. This article [Understanding Categorical Cross-Entropy Loss](https://gombru.github.io/2018/05/23/cross_entropy_loss/) covers this type of loss well.

### Generator

In [6]:
def build_generator(latent_units, vocab_size, seqlen, batchsize):
    # dense -> leakyrelu ->  batchnorm * 3 -> reshape to seqlen
    # final dense must be len(vocab_size) which we will then map to chars
    model = tf.keras.Sequential()
    
    # dense -> leakyrelu -> batchnorm
    model.add(tf.keras.layers.Dense(256, input_shape=(latent_units,)))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    # dense -> leakyrelu -> batchnorm
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    # dense -> leakyrelu -> batchnorm
    #model.add(tf.keras.layers.Dense(1024))
    #model.add(tf.keras.layers.LeakyReLU())
    #model.add(tf.keras.layers.BatchNormalization())
    
    # reshape output to seqlen
    model.add(tf.keras.layers.Dense(vocab_size*seqlen, activation='softmax'))
    
    print(model.summary())

    return model    

In [7]:
generator = build_generator(LATENT_UNITS, len(char2idx), SEQLEN, BATCHSIZE)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_2 (Dense)              (None, 6700)              3

In [8]:
generator_optimiser = tf.optimizers.Adam(1e-4)

In [9]:
def generator_loss(fake_preds):
    # take sigmoid of output FAKE predictions only
    fake_preds = tf.sigmoid(fake_preds)
    # calculate the loss with binary cross-entropy
    fake_loss = tf.losses.binary_crossentropy(tf.zeros_like(fake_preds), fake_preds)
    # return the fake predictions loss
    return fake_loss

### Discriminator

In [10]:
def build_discriminator(vocab_size, embed_dim, batchsize, units):
    model = tf.keras.Sequential()
    # add our embedding layer
    model.add(tf.keras.layers.Embedding(vocab_size, embed_dim,
                                        batch_input_shape=[batchsize, None]))
    # the LSTM layer
    model.add(tf.keras.layers.LSTM(units, return_sequences=True,
                                   stateful=True, dropout=.1))
    # a DNN layer
    model.add(tf.keras.layers.Dense(vocab_size))
    # leaky ReLU activation layer
    model.add(tf.keras.layers.LeakyReLU())

    # final binary classifier layer, 1 or 0
    model.add(tf.keras.layers.Dense(1))
  
    print(model.summary())

    return model

In [11]:
discriminator = build_discriminator(len(char2idx), EMBED_DIM, BATCHSIZE, 64)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 50)            3350      
_________________________________________________________________
lstm (LSTM)                  (64, None, 64)            29440     
_________________________________________________________________
dense_3 (Dense)              (64, None, 67)            4355      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (64, None, 67)            0         
_________________________________________________________________
dense_4 (Dense)              (64, None, 1)             68        
Total params: 37,213
Trainable params: 37,213
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
discriminator_optimiser = tf.optimizers.Adam(1e-4)

In [13]:
def discriminator_loss(real_preds, fake_preds):
    # take sigmoid of out output predictions
    real_preds = tf.sigmoid(real_preds)
    fake_preds = tf.sigmoid(fake_preds)
    # calculate the loss with binary cross-entropy
    real_loss = tf.losses.binary_crossentropy(tf.ones_like(real_preds), real_preds)
    fake_loss = tf.losses.binary_crossentropy(tf.zeros_like(fake_preds), fake_preds)
    # return the total loss from both real and fake predictions
    return real_loss + fake_loss

## Training

In [14]:
def train_step(sequences):
    fake_sequence_noise = np.random.randn(BATCHSIZE, LATENT_UNITS)
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # generate fake sequences with the generator model
        generated_sequences = generator(fake_sequence_noise)

        # get real and fake output predictions from the discriminator model
        real_output = discriminator(sequences)
        fake_output = discriminator(generated_sequences)
        
        # get the loss for both the generator and discriminator models
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
        # get the gradients for both the generator and discriminator models
        grads_generator = gen_tape.gradient(
            gen_loss, generator.trainable_variables
        )
        grads_discriminator = disc_tape.gradient(
            disc_loss, discriminator.trainable_variables
        )

In [15]:
def train(dataset, epochs):
    for _ in range(epochs):
        for sequences in dataset:
            train_step(sequences)

In [16]:
train(data, EPOCHS)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ResourceExhaustedError: OOM when allocating tensor with shape[67,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:MatMul]